In [1]:
import numpy as np
import nibabel as nib
from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table
from dipy.reconst.dti import TensorModel

# Load data same as before
print("Loading data...")
subject_path = r'diffusion_data\100206\T1w\Diffusion'
dwi_img = nib.load(f'{subject_path}/data.nii.gz')
mask_img = nib.load(f'{subject_path}/nodif_brain_mask.nii.gz')

dwi_data = dwi_img.get_fdata()
mask = mask_img.get_fdata()
print(f"DWI data shape: {dwi_data.shape}")  # Should be (X, Y, Z, num_volumes)
print(f"Mask shape: {mask.shape}")          # Should be (X, Y, Z)

# Load and create gradient table
bvals, bvecs = read_bvals_bvecs(f'{subject_path}/bvals', 
                               f'{subject_path}/bvecs')
gtab = gradient_table(bvals, bvecs)
print(f"Number of gradient directions: {len(bvals)}")

# Normalize the data by b0
print("\nNormalizing data...")
b0_mask = gtab.b0s_mask
b0_data = dwi_data[..., b0_mask]
b0_avg = np.mean(b0_data, axis=-1)
dwi_norm = dwi_data / (b0_avg[..., None] + 1e-6)
print(f"Normalized data shape: {dwi_norm.shape}")

# Fit tensors using all directions
print("\nFitting tensors...")
tensor_model = TensorModel(gtab)
tensor_fit = tensor_model.fit(dwi_norm, mask=mask)

# Extract tensor components
# DTI tensor is symmetric, so we only need 6 components
# [Dxx, Dyy, Dzz, Dxy, Dxz, Dyz]
print("\nExtracting tensor components...")
tensor_components = tensor_fit.lower_triangular()
print(f"Tensor components shape: {tensor_components.shape}")  # Should be (X, Y, Z, 6)

# Get valid voxel indices from mask
valid_idx = np.where(mask > 0)
print(f"\nNumber of valid voxels: {len(valid_idx[0])}")

# Extract tensors for valid voxels
ground_truth_tensors = tensor_components[valid_idx]
print(f"Ground truth tensors shape: {ground_truth_tensors.shape}")  # Should be (n_valid_voxels, 6)

# Basic sanity checks
print("\nSanity checks:")
print(f"Max tensor value: {np.max(ground_truth_tensors)}")
print(f"Min tensor value: {np.min(ground_truth_tensors)}")

# Optional: Save coordinates of valid voxels for reference
valid_coordinates = np.array(valid_idx).T  # Shape: (n_valid_voxels, 3)

# Save ground truth data
print("\nSaving ground truth data...")
np.savez('ground_truth.npz',
         tensors=ground_truth_tensors,
         coordinates=valid_coordinates)

Loading data...
DWI data shape: (145, 174, 145, 288)
Mask shape: (145, 174, 145)
Number of gradient directions: 288

Normalizing data...
Normalized data shape: (145, 174, 145, 288)

Fitting tensors...

Extracting tensor components...
Tensor components shape: (145, 174, 145, 6)

Number of valid voxels: 936256
Ground truth tensors shape: (936256, 6)

Sanity checks:
Max tensor value: 0.273804831545045
Min tensor value: -0.016622946229203295

Saving ground truth data...
